Descripcion proyecto


# Autenticador Drive

In [80]:
# vincular colab con Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#Librerias

In [81]:
# @title Instalar libreria
!pip install sweetviz
!pip install category_encoders
!pip install --upgrade ydata-profiling


In [83]:
# importar librerias y modulos a utilizar
import os
import re
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import sweetviz as sw
import category_encoders as ce
from google.colab import files

#ML

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.decomposition import PCA
import xgboost as xgb


#Balanceo
from imblearn.under_sampling import CondensedNearestNeighbour, EditedNearestNeighbours, InstanceHardnessThreshold, RepeatedEditedNearestNeighbours, OneSidedSelection
from imblearn.under_sampling import TomekLinks
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import NearMiss

#Algoritmos
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_text
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# Funciones

In [84]:
# @title Cargamos las funciones relacionadas
# Función para analizar la base de datos que devuelve cantidad de registros, valores únicos, total de vacíos, porcentaje de valores faltantes
def analizar_columnas(df):
  """
  Analiza las columnas de un DataFrame y devuelve información estadística.

  Parámetros:
  df (DataFrame): El DataFrame a analizar.

  Retorno:
  Un DataFrame con la información estadística de cada columna.

  Ejemplo:
  df_analizado = analizar_columnas(df)
  print(df_analizado)
  """
  info = []
  for columna in df.columns:
    unicos = df[columna].nunique()
    nulos = df[columna].isnull().sum()
    total = len(df)
    porcentaje_nulos = (nulos / total) * 100
    info.append({
      'Columna': columna,
      'Cantidad de registros': total,
      'Únicos': unicos,
      'Cantidad de nulos': nulos,
      'Porcentaje de nulos': porcentaje_nulos,
      'Registros relacionados con nulos': total - nulos
    })
  return pd.DataFrame(info)

# Ruta de datos

In [85]:
# ruta de la carpeta donde están guardadas las bases de datos
path1="/content/drive/MyDrive/TalentoTech/Datos/EDI_SECTOR_2023/"
path2="/content/drive/MyDrive/TalentoTech/Datos/EDI_SECTOR_2022/"
path3="/content/drive/MyDrive/TalentoTech/Datos/EDI_SECTOR_2021/"

# Base de datos

In [86]:
# cargamos cada una de las bases de datos
bbdd1 = pd.read_excel(path1+"base_edi_anonimizada_por_sector_2023.xlsx")
bbdd2 = pd.read_excel(path2+"base_edi_anonimizada_por_sector_2022.xlsx")
bbdd3 = pd.read_excel(path3+"base_edi_anonimizada_por_sector_2021.xlsx")


In [88]:
bbdd_final = pd.concat([bbdd1, bbdd2, bbdd3])

# Limpieza de datos

In [ ]:
# los elementos los borraron desde la base
#bbdd_final.drop(columns=['C16b', 'C16c', 'C16e', 'C16f','G03b', 'G03c', 'G03d', 'G03e', 'G03e', 'C20b', 'C21a', 'C21b', 'E08a', 'E08b', 'E08c', 'E08d'], inplace=True)

In [89]:
bbdd_final.head(3)

,Cod_grupo,Cod_sector,B02,B03,B04,B05,B06,B07,C01c,C01d,...,G07e,H04,H05,H06,Fexpfinal,Cod_Sector,E06a,E06b,E06c,E06d
0,9,5.00,5,1,1,5,32,2,5,5,...,5,1,99,99,1,NaN,NaN,NaN,NaN,NaN
1,7,22.00,4,4,1,7,61,2,5,5,...,5,2,3,2,"10,3501670764769",NaN,NaN,NaN,NaN,NaN
2,20,8.00,3,2,2,8,51,2,3,4,...,3,7,4,99,"1,05",NaN,NaN,NaN,NaN,NaN


In [67]:
# @title Verificar la forma de los datos
print(f"Forma de los datos (filas, columnas): {bbdd_final.shape}")

Forma de los datos (filas, columnas): (79629, 181)


In [68]:
# @title Verificar los tipos de datos de cada columna
print("\nTipos de datos por columna:")
print(bbdd_final.dtypes)


Tipos de datos por columna:
Cod_grupo       int64
Cod_sector    float64
B02             int64
B03             int64
B04             int64
               ...   
Cod_Sector    float64
E06a          float64
E06b          float64
E06c          float64
E06d          float64
Length: 181, dtype: object


In [69]:
#@title Contar los valores únicos por columna
print("\nValores únicos por columna:")
print(bbdd_final.nunique())


Valores únicos por columna:
Cod_grupo     15
Cod_sector    30
B02            5
B03            5
B04            2
              ..
Cod_Sector    30
E06a           6
E06b           6
E06c           6
E06d           6
Length: 181, dtype: int64


In [22]:
#@title Verificar si hay datos faltantes
print("\nDatos faltantes por columna:")
print(bbdd_final.isnull().sum())


Datos faltantes por columna:
Cod_grupo         0
Cod_sector    26824
B02               0
B03               0
B04               0
              ...  
E09a          52805
E09b          52805
E09c          52805
E09d          52805
F01d          52805
Length: 154, dtype: int64


In [90]:
# @title Verificación de los campos con relación a la composición de la base de dato almacenada en formato CSV
df_analizado = analizar_columnas(bbdd_final)
df_analizado

,Columna,Cantidad de registros,Únicos,Cantidad de nulos,Porcentaje de nulos,Registros relacionados con nulos
0,Cod_grupo,79629,15,0,0.00,79629
1,Cod_sector,79629,30,26824,33.69,52805
2,B02,79629,5,0,0.00,79629
3,B03,79629,5,0,0.00,79629
4,B04,79629,2,0,0.00,79629
...,...,...,...,...,...,...
176,Cod_Sector,79629,30,52805,66.31,26824
177,E06a,79629,6,52805,66.31,26824
178,E06b,79629,6,52805,66.31,26824
179,E06c,79629,6,52805,66.31,26824


In [91]:
df_analizado[df_analizado["Cantidad de nulos"]>10000].Columna.unique()

array(['Cod_sector', 'D11', 'E04a', 'E04b', 'E04c', 'E04d', 'F03h',
       'F03i', 'F03j', 'F05', 'F06b', 'F07f', 'Cod_Sector', 'E06a',
       'E06b', 'E06c', 'E06d'], dtype=object)

In [92]:
bbdd_final_ = bbdd_final.copy()

In [ ]:
# los elementos los borraron desde la base
bbdd_final_.drop(columns=['Cod_sector', 'D11', 'E04a', 'E04b', 'E04c', 'E04d', 'F03h',
       'F03i', 'F03j', 'F05', 'F06b', 'F07f', 'Cod_Sector', 'E06a',
       'E06b', 'E06c', 'E06d', 'Fexpfinal'], inplace=True)

In [93]:
# @title Realizamos la descriptiva estadística
bbdd_final_.describe()

,Cod_grupo,Cod_sector,B02,B03,B04,B05,B06,B07,C01c,C01d,...,G07d,G07e,H04,H05,H06,Cod_Sector,E06a,E06b,E06c,E06d
count,79629.00,52805.00,79629.00,79629.00,79629.00,79629.00,79629.00,79629.00,79629.00,79629.00,...,79629.00,79629.00,79629.00,79629.00,79629.00,26824.00,26824.00,26824.00,26824.00,26824.00
mean,9.00,15.79,3.29,3.10,1.55,7.46,45.22,1.75,3.96,3.98,...,5.09,5.15,13.27,50.39,49.54,15.84,5.60,8.56,13.69,10.85
std,4.64,9.80,1.04,1.41,0.50,1.42,10.87,0.43,0.98,1.06,...,2.25,2.42,25.83,45.64,46.45,9.74,12.16,21.55,31.41,26.80
min,1.00,1.00,1.00,1.00,1.00,1.00,18.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
25%,7.00,6.00,3.00,2.00,1.00,7.00,37.00,2.00,4.00,4.00,...,4.00,4.00,3.00,4.00,4.00,6.00,4.00,3.00,1.00,1.00
50%,9.00,14.00,3.00,3.00,2.00,8.00,45.00,2.00,4.00,4.00,...,4.00,4.00,5.00,88.00,10.00,14.00,4.00,4.00,2.00,3.00
75%,12.00,26.00,4.00,5.00,2.00,8.00,54.00,2.00,5.00,5.00,...,5.00,9.00,6.00,99.00,99.00,26.00,5.00,5.00,4.00,4.00
max,20.00,32.00,5.00,5.00,2.00,10.00,99.00,2.00,9.00,9.00,...,9.00,9.00,88.00,99.00,99.00,32.00,99.00,99.00,99.00,99.00


In [ ]:
# @title Análisis exploratorio de datos (EDA)
#Configurar las opciones de visualizacion Sweetviz
sw.config_parser.read_string("""
                              [Output_Defaults]
                              html_layout = widescreen
                              html_scale = 1.0
                              notebook_layout = widescreen
                              notebook_scale = 0.9
                              notebook_width = 100%
                              notebook_height = 700
                              [Layout]
                              show_logo = 0
                              """)
nombre = 'Analisis_desempeno_laboral'

advert_report = sw.analyze(bbdd_final, pairwise_analysis='on') # Realizamos el análisis de la base de datos inicial


#@markdown Guardar y mostrar reporte formato HTML
advert_report.show_html('EDA_df.html', )

#@markdown Descarga y abre el reporte en una nueva pestaña del navegador
almacenar_archivo = input('Alamcenar archivo Si o No: ').title()
if almacenar_archivo == 'Si':
  files.download('/content/EDA_df.html')

                                             |          | [  0%]   00:00 -> (? left)

/usr/local/lib/python3.11/dist-packages/sweetviz/graph_associations.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_dataframe[feature] = pd.Series(dtype=float)
/usr/local/lib/python3.11/dist-packages/sweetviz/graph_associations.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_dataframe[feature] = pd.Series(dtype=float)
/usr/local/lib/python3.11/dist-packages/sweetviz/graph_associations.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, 

Report EDA_df.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
Alamcenar archivo Si o No: si


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [94]:
%pip install rdt

In [95]:
bbdd_final_.D03b.value_counts()

,count
D03b,
5,42252
4,22045
3,5254
1,3860
2,3745
9,2473


In [97]:
import pandas as pd
import seaborn as sns
from rdt import HyperTransformer
from sklearn.model_selection import train_test_split

# 3️⃣ Eliminar filas con valores nulos
bbdd_final_ = bbdd_final_.dropna()

# 4️⃣ Separar variable objetivo
target_column = "D03b"
X = bbdd_final_.drop(columns=[target_column])
y = bbdd_final_[target_column]


# 5️⃣ Crear el transformador RDT 🚀
ht = HyperTransformer()

# 6️⃣ Detectar configuración automáticamente ✅
ht.detect_initial_config(X)


# 7️⃣ Ajustar y transformar los datos 🔄
ht.fit(X)
X_transformed = ht.transform(X)

# 8️⃣ Convertir a DataFrame para visualizar 🔍
X_transformed = pd.DataFrame(X_transformed, columns=X.columns)

# 🔟 Separar en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [24]:
! pip install lazypredict

In [32]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from lazypredict.Supervised import LazyClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

models = {
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'SVC': SVC()
}

# Entrenar y evaluar cada modelo
for name, model in models.items():
    print(f"Entrenando {name}...")
    model.fit(X_train, y_train)  # Entrenar el modelo
    y_pred = model.predict(X_test)  # Realizar predicciones

    # Evaluar el modelo
    print(f"Resultados para {name}:")
    print(classification_report(y_test, y_pred))
    print("-" * 50)

Entrenando Random Forest...


NameError: name 'X_train' is not defined